In [1]:
from dash import Dash, dcc, html, Input, Output
import grakel
from grakel.kernels import WeisfeilerLehman, VertexHistogram, ShortestPath, SubgraphMatching
import json
import os
import plotly.express as px
from rdflib import Graph, URIRef
from rdflib.namespace import RDF, SOSA
from rdflib.extras.external_graph_libs import rdflib_to_networkx_digraph
from tqdm import tqdm
import time
from viscars.namespace import DASHB

In [2]:
GRAPH_INPUT_DIR = 'data/protego'
GRAPH_SIMILARITY_DIR = '/home/pieter/OpenKE/benchmarks/DASHB'

graph = Graph()
graph.parse(os.path.join(GRAPH_INPUT_DIR, 'protego_zplus.ttl'))
graph.parse(os.path.join(GRAPH_INPUT_DIR, 'graph.ttl'))
items = graph.subjects(RDF.type, SOSA.ObservableProperty)

### Patients (Context)

In [3]:
cc_similarities = {}
with open(os.path.join(GRAPH_SIMILARITY_DIR, 'results', 'RDF2VecUU.json')) as input_f:
    cc_similarities = json.load(input_f)

### ObservableProperties (Item)

In [4]:
qry = '''
    PREFIX dashb: <http://dynamicdashboard.ilabt.imec.be/broker/ontologies/dashboard#>
    PREFIX sosa: <http://www.w3.org/ns/sosa/>
    
    SELECT ?property ?metric WHERE {
        ?property a sosa:ObservableProperty ;
            dashb:produces ?metric .
    }
'''
properties = {property_: metric for property_, metric in graph.query(qry)}

ii_similarities = {}
# with open(os.path.join(GRAPH_SIMILARITY_DIR, 'results', 'RDF2VecII.json')) as input_f:
#     ii_similarities = json.load(input_f)

### Dashboard Users (User)

In [5]:
qry = '''
    PREFIX dashb: <https://docs.dynamicdashboard.ilabt.imec.be/ontologies/dashboard#>
    
    SELECT ?user ?label ?role WHERE {
        ?user a dashb:User ;
            rdfs:label ?label ;
            dashb:memberOf ?role .
    }
'''
users = list(graph.query(qry))

uu_similarities = {}
for user, label, role in users:
    uu_similarities[user] = {}
    
    for user_, label_, role_ in users:
        if role == role_:
            uu_similarities[user][user_] = 1
        else:
            uu_similarities[user][user_] = 0

(rdflib.term.URIRef('https://dynamicdashboard.ilabt.imec.be/users/10'), rdflib.term.Literal('zplus_operator3'), rdflib.term.URIRef('https://dynamicdashboard.ilabt.imec.be/groups/operator'))


In [6]:
def kneighbours_cc(uid, k=10):
    cc_similarities_ = cc_similarities.copy()
    cc_similarities_[uid].pop(uid)
    return sorted(cc_similarities_[uid].items(), key=lambda x: x[1])[:k]

def kneighbours_ii(iid, k=10):
    ii_similarities_ = ii_similarities.copy()
    ii_similarities_[iid].pop(uid)
    return sorted(ii_similarities_[iid].items(), key=lambda x: x[1])[:k]

def kneighbours_uu(uid, k=10):
    uu_similarities_ = uu_similarities.copy()
    uu_similarities_[uid].pop(uid)
    return sorted(uu_similarities_[uid].items(), key=lambda x: x[1])[:k]

In [9]:
def predict_uucf(uid, cid, iid):
    neighbours = kneighbours_uu(uid)
    
    for n, w in neighbours:
        print(n, iid)
        qry = f'''
            PREFIX dashb: <https://docs.dynamicdashboard.ilabt.imec.be/ontologies/dashboard#>
    
            SELECT * WHERE {{
                ?widget a dashb:Widget ;
                    dashb:createdBy <{n}> ;
                    dashb:hasProperty <{iid}> .
            }}
        '''
        print(list(graph.query(qry)))

In [11]:
predict_uucf('https://dynamicdashboard.ilabt.imec.be/users/10', 'http://example.com/tx/patients/zplus_6', list(properties.items())[0][0])

KeyError: 'https://dynamicdashboard.ilabt.imec.be/users/10'